In [1]:
# Assembling RoM and PoP for option chains

# STATUS: Incomplete

import pandas as pd
import datetime
from scipy.stats import norm
import numpy as np
import math


#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
today = datetime.datetime.now().date()

#*****            Set up the Limits         ****
#_______________________________________________

# Standard Deviation limits
call_probability = 0.97  # for Calls
put_probability = 0.95   # for Puts
sd_days = 252 # no of days for the standard deviation

call_sd = norm.ppf(1-(1-call_probability)/2)
put_sd = norm.ppf(1-(1-put_probability)/2)

# Read the pickles
df_ohlc = pd.read_pickle(datapath+'df_ohlc.pkl')
df_options = pd.read_pickle(datapath+'df_nse_options.pkl')
df_nse_ib = pd.read_pickle(datapath+ 'df_underlying.pkl')

#******      Get the Annual Standard Deviations  ****
#____________________________________________________

# get the max and min of ohlc for each date
df_ohlc['maxp'] = df_ohlc.loc[:, ['O', 'H', 'L', 'C']].max(1) # Max price for calls
df_ohlc['minp'] = df_ohlc.loc[:, ['O', 'H', 'L', 'C']].min(1) # Min price for puts

# Compute the annual standard deviation for calls and puts
df_cSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'maxp'].std(ddof=0).rename(columns={'maxp': 'cASD'})

df_pSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'minp'].std(ddof=0).rename(columns={'minp': 'pASD'})

df_aSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'C'].std(ddof=0).rename(columns={'C': 'ASD'})

df_SD = pd.concat(objs=[df_aSD, df_cSD, df_pSD], axis=1).reset_index()

In [58]:
# Convert Expiry column into datetime
df_options.Expiry = pd.to_datetime(df_options.Expiry)

# get the days to expiry for the options
df_options['DTE'] = (df_options.Expiry - datetime.datetime.now()).dt.days

# Determine put or call - option type
df_options['Type'] = np.where(df_options.Strike > df_options.undPrice, 'c', 'p')

In [3]:
# Compute risk-free rate (r) from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
r = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100

In [4]:
#*****      European Black-Scholes     ******
#____________________________________________

# Ref: https://github.com/dedwards25/Python_Option_Pricing/blob/master/GBS.ipynb

def get_eu_bs(option_type, fs, x, t, r, v, b=0):
    '''Gets the Black-Scholes option prices and deltas
    Args:
       (option_type): 'p' for put | 'c' for call
       (fs): underlying price in float
       (x): strike price in float
       (t): time to expiration in int days
       (r): risk-free-rate in float
       (b): cost-of-carry or dividend in float. Default = 0
       (v): implied volatility or annual standard deviation in float
    Returns:
       (value, delta, gamma, theta, vega, rho) tuple
    '''
     # -----------
    # Create preliminary calculations
    t__sqrt = math.sqrt(t)
    d1 = (math.log(fs / x) + (b + (v * v) / 2) * t) / (v * t__sqrt)
    d2 = d1 - v * t__sqrt

    if option_type == "c":
        # it's a call
        value = fs * math.exp((b - r) * t) * norm.cdf(d1) - x * math.exp(-r * t) * norm.cdf(d2)
        delta = math.exp((b - r) * t) * norm.cdf(d1)
        gamma = math.exp((b - r) * t) * norm.pdf(d1) / (fs * v * t__sqrt)
        theta = -(fs * v * math.exp((b - r) * t) * norm.pdf(d1)) / (2 * t__sqrt) - (b - r) * fs * math.exp(
            (b - r) * t) * norm.cdf(d1) - r * x * math.exp(-r * t) * norm.cdf(d2)
        vega = math.exp((b - r) * t) * fs * t__sqrt * norm.pdf(d1)
        rho = x * t * math.exp(-r * t) * norm.cdf(d2)
    else:
        # it's a put
        value = x * math.exp(-r * t) * norm.cdf(-d2) - (fs * math.exp((b - r) * t) * norm.cdf(-d1))
        delta = -math.exp((b - r) * t) * norm.cdf(-d1)
        gamma = math.exp((b - r) * t) * norm.pdf(d1) / (fs * v * t__sqrt)
        theta = -(fs * v * math.exp((b - r) * t) * norm.pdf(d1)) / (2 * t__sqrt) + (b - r) * fs * math.exp(
            (b - r) * t) * norm.cdf(-d1) + r * x * math.exp(-r * t) * norm.cdf(-d2)
        vega = math.exp((b - r) * t) * fs * t__sqrt * norm.pdf(d1)
        rho = -x * t * math.exp(-r * t) * norm.cdf(-d2)
    
    return (value, delta, gamma, theta, vega, rho)

# Add ibSymbol, margin, lots and standard deviations to df_options
df_nse_ib = pd.read_pickle(datapath+ 'df_underlying.pkl') # Link between nse and ib symbols

df = df_options.merge(df_nse_ib, on='nseSymbol').merge(df_SD, on='ibSymbol')

# Add risk-free rate 
df['Rate'] = r

In [5]:
get_eu_bs(option_type='c', fs=100, x=88, t=10, r=0.06, v=20.66, b=0)

(54.88116360940265,
 0.5488116360940265,
 6.053567152093668e-237,
 3.292869816564159,
 1.2506669736225516e-230,
 1.2096543316569679e-231)

In [65]:
list(df)

['nseSymbol',
 'Expiry',
 'Strike',
 'undPrice',
 'cOI',
 'cChnginOI',
 'cVolume',
 'cIV',
 'cLTP',
 'cNetChng',
 'cBidQty',
 'cBidPrice',
 'cAskPrice',
 'cAskQty',
 'pBidQty',
 'pBidPrice',
 'pAskPrice',
 'pAskQty',
 'pNetChng',
 'pLTP',
 'pIV',
 'pVolume',
 'pChnginOI',
 'pOI',
 'DTE',
 'Type',
 'Mlot',
 'marginpct',
 'ibSymbol',
 'ASD',
 'cASD',
 'pASD',
 'Rate']

In [67]:
df1 = df.loc[:, ['nseSymbol', 'Type', 'undPrice', 'Strike', 'DTE', 'Rate', 'ASD']]
df1.DTE = pd.to_numeric(df1.DTE)

In [16]:
# Compute the bsm
# Add Model value and Delta (PoP)
# Compute RoM 

In [80]:
list(df1.loc[32])

['ACC', 'p', 1439.15, 1140.0, 29, 0.068121, 107.43938469592979]

In [91]:
get_eu_bs('c', 10658, 10550, 29, 0.06, 0.2087*10658)

(1870.6964297759525, 0.17552040061699686, 0.0, 112.24178578655716, 0.0, 0.0)

In [ ]:
np.vectorize(get_eu_bs(option_type=df.Type, fs=df.undPrice, x=df.Strike, t=df.DTE, r=df.Rate, v=df.ASD, b=0))

In [ ]:
df.apply(lambda x: get_eu_bs(x.Type, x.undPrice, x.Strike, x.DTE, x.Rate, x.ASD))